In [ ]:
import base64
import io
#Global Stuff, usually this can just be ran without issues
import re
import re
from pprint import pprint

import requests
from PIL import Image, PngImagePlugin

url = "http://127.0.0.1:7860"  # Might need to change to your Stablediffusion UI Hostname/IP


In [ ]:
def generate_image(payload, outfile):
    """
    This function generates images and displays them inline in the notebook
    """
    response = requests.post(url=f'{url}/sdapi/v1/txt2img', json=payload)

    r = response.json()

    for i in r['images']:
        image = Image.open(io.BytesIO(base64.b64decode(i.split(",", 1)[0])))
        from pprint import pprint
        #from IPython import display
        scale = 0.5
        scale = 1
        display(image.resize((int(image.width * scale), int(image.height * scale))))


def set_model(model=None):
    """
    This function sets the model to something provided
    If nothing is provided, will just return true
    """
    if model:
        print(f"Change to model {model}")
        option_payload = {
            "sd_model_checkpoint": model
        }
        response = requests.post(url=f'{url}/sdapi/v1/options', json=option_payload)
        if response.status_code == 200:
            return True
        else:
            print("Failed to set model")
            return False
    else:
        print("No Model Change necessary")
        return True


def generate_images(samplers, payload, model=None, artists=None, keyword_test=None):
    """
    This function deals with the iterations of images
    Model: None = Model will not be changed
    artists: None = No need to check artists
    keyword_test = None: Will not test for tags, set to something if you want to test in combination with tags
    """
    regexpattern = r"(.*)?_(\d+)?_training_images_(\d+)?_"
    prompt = payload["prompt"]
    model_name, pictures, training_steps = re.search(regexpattern, model).groups()
    if set_model(model):
        for sampler in samplers:
            if sampler == "LMS Karras" and steps >= 90:
                print("Skipping LMS Karras, only noise from Step 90+")
            else:
                if artists:
                    for artist in artists:
                        payload["prompt"] = prompt + ", by " + artist
                        generate_image(payload, f"{payload['seed']}_{sampler}")
                        print(
                            f"Artist: '{artist}' Model Name: '{model_name}' Picture Count: {pictures} Training Steps: {training_steps}\nSampler: '{sampler}' Steps: {payload['steps']} Seed: {payload['seed']}")
                elif keyword_test:
                    keywords = payload["prompt"].split(", ")
                    for keyword in keywords:
                        payload["prompt"] = keyword_test + ", " + keyword
                        generate_image(payload, f"{payload['seed']}_{sampler}")
                        print(
                            f"Keyword: '{keyword}' Model Name: '{model_name}' Picture Count: {pictures} Training Steps: {training_steps}\nSampler: '{sampler}' Steps: {payload['steps']} Seed: {payload['seed']}")
                else:
                    payload["sampler_name"] = sampler
                    generate_image(payload, f"{payload['seed']}_{sampler}")
                    print(
                        f"Model Name: '{model_name}' Picture Count: {pictures} Training Steps: {training_steps}\nSampler: '{sampler}' Steps: {payload['steps']} Seed: {payload['seed']}")
        print("done")

In [ ]:
def initialise(seed, prompt, negative_prompt, steps, samplers=None):
    """
    Function generates a default payload, and sets default sampler list if none is provided
    """
    if seed == -1:
        import random
        seed = random.randint(1000000000, 10000000000)
    payload = {
        "enable_hr": False,
        #"denoising_strength": 0,
        #"firstphase_width": 0,
        #"firstphase_height": 0,
        "prompt": prompt,
        "negative_prompt": negative_prompt,
        #"styles": [
        #  "Good Baseline NSFW",
        #  "Good Baseline NSFW"
        #],
        "seed": seed,
        #"subseed": -1,
        #"subseed_strength": 0,
        #"seed_resize_from_h": -1,
        #"seed_resize_from_w": -1,
        #"sampler_name": "string",
        "batch_size": 1,
        "n_iter": 1,
        "steps": steps,
        "cfg_scale": 7,
        "width": 512,
        "height": 512,
        "restore_faces": False,
        "tiling": False,
        #"eta": 0,
        #"s_churn": 0,
        #"s_tmax": 0,
        #"s_tmin": 0,
        #"s_noise": 1,
        #"override_settings": {},
        "sampler_index": "Euler a"
    }

    if not samplers:
        samplers = [
            "Euler a",
            "Euler",
            "LMS",
            "Heun",
            "LMS Karras",
            "DPM fast",
            "DPM adaptive",
            "DPM2",
            "DPM2 a",
            "DPM2 Karras",
            "DPM2 a Karras",
            "DPM++ 2S a Karras",
            "DPM++ 2M Karras",
            "DPM++ 2S a",
            "DPM++ 2M",
            "DDIM",
            "PLMS"
        ]
    return payload, samplers

### Model Tester

In [ ]:
# Some Prompt you want to use for Model Testing
prompt = "linus_wd1, pissed off, annoyed, upset, angry, plantigrade, photorealistic, natural lighting, masterpiece, high quality, hi res, highres"
# Some Negative Prompt you want to use for Model Testing
negative_prompt = "deformed, blurry, bad, disfigured, poor, mutation, mutated, ugly, horror, out of focus, depth of field, focal blur, worst quality, low quality, normal quality, deformed, blurry, bad, disfigured, poor, mutation, mutated, ugly, horror, out of focus, depth of field, focal blur, worst quality, low quality, normal quality"
# Seed, if set to -1 autogenerate will be done
seed = -1
# How many Steps you want
steps = 90
# Which samplers you want to check
# If none are supplied, it will default to all
samplers = ["Euler a",
            "DPM++ 2S a Karras",
            "DPM++ 2M Karras",
            "DPM++ 2S a",
            "DPM++ 2M", ]
# Names of the models you want to check in an array
models = ["WildDogOne_10_training_images_2000_max_training_steps_linus_wd1_token_anthro_class_word.ckpt [2fff1efe]",
          "wildDogOne_8_training_images_2000_max_training_steps_linus_wd1_token_anthro_class_word.ckpt [999fb79e]",
          "WildDogOne_8_training_images_2500_max_training_steps_linus_wd1_token_anthro_class_word.ckpt [eaa4154a]"]
# Get Payload and Sampler information from initialise function
# You can also just do that by hand
payload, samplers = initialise(seed, prompt, negative_prompt, steps, samplers)

# Generate Images for each model and sampler
for model in models:
    generate_images(samplers, payload, model)

### Artist Tester

In [ ]:
# Array of artist you want to test against
artists = ["Michael & Inessa Garmash",
           "Ruan Jia",
           "Edmund Dulac",
           "Rumiko Takahashi",
           "Eileen Agar",
           "Katsushika Hokusai",
           "Hokusai",
           "Nicolas Delort",
           "Andy Kehoe",
           "Conrad Roset",
           "Helen Frankenthaler",
           "Giotto Di Bondone",
           "Takashi Murakami",
           "Eduardo Paolozzi",
           "Rebecca Louise Law",
           "Harumi Hironaka",
           "Alexander Calder",
           "Charles Blackman",
           "Jeremiah Ketner",
           "Jim Mahfood",
           "sepulte",
           "pixelsketcher",
           "inubashiri",
           "personalami",
           "chunie",
           "Pino Daeni",
           "Kentaro Miura",
           "Carl Larsson",
           "Bernard Buffet",
           "Don Bluth",
           "Naoko Takeuchi",
           "Ethan Van Sciver",
           "Barry Windsor Smith",
           "Milton Caniff",
           "Osamu Tezuka",
           "Toei Animations", ]

# Sampler to use
# I usually only take 1-2 different samplers
samplers = ["Euler a"]
# The Prompt to use with the artists
prompt = "linus_wd1, plantigrade, photorealistic, natural lighting, masterpiece, high quality, hi res, highres"
# The Negative Prompt to use with the artists
negative_prompt = "deformed, blurry, bad, disfigured, poor, mutation, mutated, ugly, horror, out of focus, depth of field, focal blur, worst quality, low quality, normal quality, deformed, blurry, bad, disfigured, poor, mutation, mutated, ugly, horror, out of focus, depth of field, focal blur, worst quality, low quality, normal quality"
# Seed, if -1 it will be random
seed = 4805326764
# How many steps
steps = 20
# Which model to use, if None, it will just use whatever is loaded
model = "WildDogOne_10_training_images_2000_max_training_steps_linus_wd1_token_anthro_class_word.ckpt [2fff1efe]"

# Get Payload and Sampler information from initialise function
# You can also just do that by hand
payload, samplers = initialise(seed, prompt, negative_prompt, steps, samplers)

generate_images(samplers, payload, model=model, artists=artists)

### Prompt Tester

In [ ]:
# Prompt checker
# Prompts to use for the prompt checker
# This string will be split by ", " and these prompts will be evaluated
prompt = "plantigrade, photorealistic, natural lighting, masterpiece, high quality, uploaded on e621, anthro, detailed fluffy fur, hi res, highres, (detailed eyes), highly detailed fur, hi_res"
# Negative Prompts to add
negative_prompt = "deformed, blurry, bad, disfigured, poor, mutation, mutated, ugly, horror, out of focus, depth of field, focal blur, worst quality, low quality, normal quality, deformed, blurry, bad, disfigured, poor, mutation, mutated, ugly, horror, out of focus, depth of field, focal blur, worst quality, low quality, normal quality"
# How many steps you want to take
steps = 60
# Seed, if -1 it will be random
seed = 4205326764
# Sampler to use
# I usually only take 1-2 different samplers
samplers = ["Euler a"]
# Which model to use, if None, it will just use whatever is loaded
model = "WildDogOne_10_training_images_2000_max_training_steps_linus_wd1_token_anthro_class_word.ckpt [2fff1efe]"

# Get Payload and Sampler information from initialise function
# You can also just do that by hand
payload, samplers = initialise(seed, prompt, negative_prompt, steps, samplers)

generate_images(samplers, payload, model=model, artists=None, keyword_test="linus_wd1")